## Carrying Capacity
---


In [1]:
import matplotlib
import numpy as np
import pandas as pd
import datetime as dt
import theseus_growth as thg
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

user_set = np.array([[1,2,3], [2,3,4], [2,3,5],[2,3,6], [2,3,1], [2,3,5,6], [2,3,1]], dtype=object)

users_df = pd.DataFrame([user_set]).T
users_df.columns = ['userId']
users_df['userId']=users_df.userId.astype('object')
users_df['month'] = list(range(1,len(user_set)+1))
users_df['serviceId'] = 1


users_df['cum_userId'] = users_df.groupby('serviceId')['userId'].apply(lambda x: x.cumsum())

# users_df = pd.DataFrame(example_df.groupby(['serviceId', 'month'])['userId'].apply(list)).reset_index().copy()
# users_df['userId'] = users_df['userId'].apply(lambda d: d if isinstance(d, list) else [])

users_df['cum_userId'] = users_df['cum_userId'].apply(set)
users_df['userId'] = users_df['userId'].apply(set)

users_df['cum_userId_shifted'] = users_df.groupby('serviceId')['cum_userId'].shift(1)
users_df['userId_shifted'] = users_df.groupby('serviceId')['userId'].shift(1)

users_df['new_userId'] = users_df['cum_userId'] - users_df['cum_userId_shifted']
users_df['churn_userId'] = users_df['userId_shifted'] - users_df['userId']


# display(users_df[users_df.churn_userId.isna()])
users_df['churn_userId'] = users_df['churn_userId'].apply(lambda d: d if isinstance(d, set) else set())\
                                                    .apply(list)

users_df['cum_churn_userId'] = users_df.groupby('serviceId')['churn_userId'].apply(lambda x: x.cumsum())
users_df['churn_userId'] = users_df['churn_userId'].apply(set)
users_df['cum_churn_userId'] = users_df['cum_churn_userId'].apply(set)

users_df['returned_userId'] = users_df.apply(lambda x: x['cum_churn_userId'].intersection(x['userId']), axis=1)
users_df['current_userId'] = users_df['userId']

# display(users_df.head(4))

users_df['num_current_userId'] = users_df.current_userId.str.len()
users_df['num_new_userId'] = users_df.new_userId.str.len()
users_df['num_churn_userId'] = users_df.churn_userId.str.len()
users_df['num_returned_userId'] = users_df.returned_userId.str.len()
users_df[['current_userId','churn_userId','new_userId','returned_userId']]

/tmp/ipykernel_1073732/3252179883.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  users_df['cum_userId'] = users_df.groupby('serviceId')['userId'].apply(lambda x: x.cumsum())
/tmp/ipykernel_1073732/3252179883.py:29: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  users_df['cum_churn_userId'] 

,current_userId,churn_userId,new_userId,returned_userId
0,"{1, 2, 3}",{},NaN,{}
1,"{2, 3, 4}",{1},{4},{}
2,"{2, 3, 5}",{4},{5},{}
3,"{2, 3, 6}",{5},{6},{}
4,"{1, 2, 3}",{6},{},{1}
5,"{2, 3, 5, 6}",{1},{},"{5, 6}"
6,"{1, 2, 3}","{5, 6}",{},{1}


In [3]:
# # ref. https://stackoverflow.com/questions/45239540/intersection-of-sets-as-columns-in-pandas
# df=pd.DataFrame([[set([1,2,3,4]), set([1,2,3,4]), set([1,2,3,4]),set([1,2,3,4])], 
#                   [set([2,3]), set([1]), set([4]),set([3,4])]]).T

# df.columns = ['i','j']

# df['i-j'] = df['i'] - df['j']
# df['k'] = df.apply(lambda x: x['i'].intersection(x['j']), axis=1)
# df['len_j'] = df.j.str.len()
# df

## Cohort Analysis
--- 


In [4]:
cm = sns.color_palette("RdPu", n_colors=22, as_cmap=True)

df = pd.read_csv('data/scanner_data.csv')
df.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


In [5]:
df = df.drop(['Unnamed: 0'], axis = 1)
df.Date = pd.to_datetime(df.Date, format='%d/%m/%Y')
df.duplicated(["Date","Customer_ID"]).sum()

68979

In [6]:
df.describe()

,Customer_ID,Transaction_ID,Quantity,Sales_Amount
count,131706.000000,131706.000000,131706.000000,131706.000000
mean,12386.450367,32389.604187,1.485311,11.981524
std,6086.447552,18709.901238,3.872667,19.359699
min,1.000000,1.000000,0.010000,0.020000
25%,7349.000000,16134.000000,1.000000,4.230000
50%,13496.000000,32620.000000,1.000000,6.920000
75%,17306.000000,48548.000000,1.000000,12.330000
max,22625.000000,64682.000000,400.000000,707.730000


In [7]:
df = pd.DataFrame(df.groupby(["Date","Customer_ID"]).agg({'Transaction_ID':max
                                                          , 'SKU_Category':max, 'SKU':max, 'Quantity': sum
                                                          , 'Sales_Amount':sum})).reset_index()
df.head()

,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,2016-01-02,3,90,TW8,Y1M2E,4.0,10.92
1,2016-01-02,178,84,R6E,HO1M5,2.0,58.99
2,2016-01-02,195,107,LGI,VY2UB,2.0,13.10
3,2016-01-02,343,134,XG4,ZSVWE,1.0,6.75
4,2016-01-02,399,136,P42,XJLWY,2.0,10.43


In [8]:
def diff_month(x):
    d1 = x['tx_month']
    d2 = x["acq_month"]
    return ((d1.year - d2.year) * 12 + d1.month - d2.month)+1

def get_cohort_matrix(data, var='Customer_ID', fun=pd.Series.nunique):
    cd = data.groupby(['acq_month', 'cohort_idx'])[var].apply(fun).reset_index()
    cc = cd.pivot_table(index = 'acq_month',
                        columns = 'cohort_idx',
                        values = var)
    cs = cc.iloc[:,0]
    retention = cc.divide(cs, axis = 0)
    retention = retention.round(3) * 100
    return cc, retention

In [9]:
df['tx_month'] = df['Date'].apply(lambda x: dt.date(x.year, x.month,1))
df['acq_month'] = df.groupby('Customer_ID')['tx_month'].transform('min')
df['cohort_idx'] = df.apply(lambda x: diff_month(x), axis=1)

In [10]:
cc, retention = get_cohort_matrix(df)
display(cc.style.background_gradient(cmap=cm, axis=None).format(precision=1))
display(retention.style.background_gradient(cmap=cm, axis=None).format(precision=1))

cohort_idx,1,2,3,4,5,6,7,8,9,10,11,12
acq_month,,,,,,,,,,,,
2016-01-01,3396.0,1306.0,1025.0,597.0,266.0,197.0,152.0,137.0,90.0,56.0,41.0,40.0
2016-02-01,2221.0,478.0,291.0,147.0,95.0,68.0,68.0,57.0,22.0,18.0,14.0,nan
2016-03-01,2642.0,746.0,651.0,600.0,530.0,521.0,530.0,512.0,513.0,532.0,nan,nan
2016-04-01,2527.0,728.0,633.0,553.0,566.0,595.0,566.0,567.0,581.0,nan,nan,nan
2016-05-01,2626.0,626.0,559.0,505.0,566.0,551.0,521.0,523.0,nan,nan,nan,nan
2016-06-01,1569.0,278.0,274.0,280.0,278.0,277.0,300.0,nan,nan,nan,nan,nan
2016-07-01,1213.0,172.0,183.0,189.0,182.0,201.0,nan,nan,nan,nan,nan,nan
2016-08-01,1060.0,163.0,138.0,143.0,126.0,nan,nan,nan,nan,nan,nan,nan
2016-09-01,1625.0,309.0,263.0,297.0,nan,nan,nan,nan,nan,nan,nan,nan


cohort_idx,1,2,3,4,5,6,7,8,9,10,11,12
acq_month,,,,,,,,,,,,
2016-01-01,100.0,38.5,30.2,17.6,7.8,5.8,4.5,4.0,2.7,1.6,1.2,1.2
2016-02-01,100.0,21.5,13.1,6.6,4.3,3.1,3.1,2.6,1.0,0.8,0.6,nan
2016-03-01,100.0,28.2,24.6,22.7,20.1,19.7,20.1,19.4,19.4,20.1,nan,nan
2016-04-01,100.0,28.8,25.0,21.9,22.4,23.5,22.4,22.4,23.0,nan,nan,nan
2016-05-01,100.0,23.8,21.3,19.2,21.6,21.0,19.8,19.9,nan,nan,nan,nan
2016-06-01,100.0,17.7,17.5,17.8,17.7,17.7,19.1,nan,nan,nan,nan,nan
2016-07-01,100.0,14.2,15.1,15.6,15.0,16.6,nan,nan,nan,nan,nan,nan
2016-08-01,100.0,15.4,13.0,13.5,11.9,nan,nan,nan,nan,nan,nan,nan
2016-09-01,100.0,19.0,16.2,18.3,nan,nan,nan,nan,nan,nan,nan,nan


In [11]:
cc_q, ret_q = get_cohort_matrix( df, var='Quantity', fun=pd.Series.mean )
cc_sa, ret_sa = get_cohort_matrix( df, var='Sales_Amount', fun=pd.Series.median )

display(cc_q.style.background_gradient(cmap=cm, axis=None).format(precision=1))
display(ret_q.style.background_gradient(cmap=cm, axis=None).format(precision=1))
display(cc_sa.style.background_gradient(cmap=cm, axis=None).format(precision=1))
display(ret_sa.style.background_gradient(cmap=cm, axis=None).format(precision=1))

cohort_idx,1,2,3,4,5,6,7,8,9,10,11,12
acq_month,,,,,,,,,,,,
2016-01-01,3.2,3.3,3.1,3.0,3.5,3.6,3.6,2.9,3.9,3.0,6.0,2.5
2016-02-01,3.0,2.7,3.1,2.8,3.1,3.8,2.4,2.5,6.6,1.6,1.8,nan
2016-03-01,2.9,3.4,3.8,3.6,3.9,3.6,3.7,3.8,3.4,3.4,nan,nan
2016-04-01,2.8,3.3,3.5,2.9,3.3,3.3,3.6,3.1,3.5,nan,nan,nan
2016-05-01,2.7,3.2,3.2,3.1,3.1,3.2,3.6,3.3,nan,nan,nan,nan
2016-06-01,2.7,3.1,3.4,2.6,3.2,2.7,3.5,nan,nan,nan,nan,nan
2016-07-01,2.7,2.5,2.7,3.5,2.7,3.3,nan,nan,nan,nan,nan,nan
2016-08-01,2.6,2.8,2.7,3.6,2.6,nan,nan,nan,nan,nan,nan,nan
2016-09-01,2.9,3.2,3.7,3.6,nan,nan,nan,nan,nan,nan,nan,nan


cohort_idx,1,2,3,4,5,6,7,8,9,10,11,12
acq_month,,,,,,,,,,,,
2016-01-01,100.0,105.7,99.0,96.2,110.9,115.2,115.0,90.3,121.7,94.5,189.6,78.8
2016-02-01,100.0,91.1,102.6,94.3,103.4,127.9,79.2,83.2,222.3,55.3,61.1,nan
2016-03-01,100.0,114.9,128.6,124.2,134.4,124.6,128.4,129.8,117.2,117.1,nan,nan
2016-04-01,100.0,117.7,125.6,101.6,116.2,116.2,127.7,111.2,123.0,nan,nan,nan
2016-05-01,100.0,116.6,119.4,114.1,114.8,118.6,133.7,123.5,nan,nan,nan,nan
2016-06-01,100.0,113.1,125.5,95.0,116.7,100.5,127.8,nan,nan,nan,nan,nan
2016-07-01,100.0,90.4,97.9,129.3,100.4,119.4,nan,nan,nan,nan,nan,nan
2016-08-01,100.0,105.8,103.5,137.6,100.7,nan,nan,nan,nan,nan,nan,nan
2016-09-01,100.0,109.3,126.5,123.5,nan,nan,nan,nan,nan,nan,nan,nan


cohort_idx,1,2,3,4,5,6,7,8,9,10,11,12
acq_month,,,,,,,,,,,,
2016-01-01,11.5,13.3,12.7,12.5,14.7,15.7,13.7,12.1,12.1,11.2,11.8,13.8
2016-02-01,12.1,12.4,12.1,11.2,14.3,14.7,11.1,9.9,12.2,9.9,11.9,nan
2016-03-01,12.0,13.6,13.0,13.8,14.3,13.1,14.4,16.2,13.8,14.9,nan,nan
2016-04-01,11.9,12.6,13.4,12.6,13.2,13.4,13.4,13.8,14.3,nan,nan,nan
2016-05-01,11.6,13.5,12.6,13.1,12.2,12.2,13.4,13.1,nan,nan,nan,nan
2016-06-01,12.3,12.7,11.9,11.5,11.7,12.6,13.5,nan,nan,nan,nan,nan
2016-07-01,12.0,11.6,12.1,12.7,13.8,13.6,nan,nan,nan,nan,nan,nan
2016-08-01,11.9,10.0,11.4,12.2,11.6,nan,nan,nan,nan,nan,nan,nan
2016-09-01,11.9,10.7,12.4,13.8,nan,nan,nan,nan,nan,nan,nan,nan


cohort_idx,1,2,3,4,5,6,7,8,9,10,11,12
acq_month,,,,,,,,,,,,
2016-01-01,100.0,116.0,110.6,108.8,128.1,137.3,119.5,105.1,105.7,97.4,102.8,120.0
2016-02-01,100.0,102.4,100.0,92.7,118.1,121.8,91.5,82.1,100.7,81.7,98.9,nan
2016-03-01,100.0,113.8,108.5,114.9,119.4,109.5,120.1,135.4,115.4,123.9,nan,nan
2016-04-01,100.0,106.4,113.3,106.5,111.4,113.2,113.3,116.4,120.5,nan,nan,nan
2016-05-01,100.0,116.8,108.8,113.0,105.2,104.9,115.7,113.2,nan,nan,nan,nan
2016-06-01,100.0,102.9,96.8,93.3,94.9,102.6,109.5,nan,nan,nan,nan,nan
2016-07-01,100.0,96.2,100.4,105.7,114.3,113.0,nan,nan,nan,nan,nan,nan
2016-08-01,100.0,84.3,95.8,102.7,97.8,nan,nan,nan,nan,nan,nan,nan
2016-09-01,100.0,90.0,104.2,115.5,nan,nan,nan,nan,nan,nan,nan,nan


## References

### Carrying Capacity
---
- https://playinpap.github.io/advanced-carrying-capacity/
- pandas 
    - https://stackoverflow.com/questions/44719855/cumulative-list-of-a-column-using-groupby
    - https://stackoverflow.com/questions/45239540/intersection-of-sets-as-columns-in-pandas


### Cohort Analysis
---
- https://www.activestate.com/blog/cohort-analysis-with-python/
- https://www.slideshare.net/leoyang991/ss-90038927
- code & data from
    - https://github.com/nickmancol/python-cohorts
